<a href="https://colab.research.google.com/github/AnitaKirkovska/Quantum_Computing/blob/master/HW1/hw_1_problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW 1 ##

Sources used: Github

### Problem 2 ###

**Some definitions**

Let 

$$M\in\mathbb{R}^{n\times n}$$ be an arbitrary matrix.  

Let $$p(x)=a_0 + a_1 x + a_2 x^2 + \ldots + a_n x^n \in\mathbb{R}[x]$$ be an arbitrary polynomial of less or equal to $n$.

The above polynomial can be used to define a matrix function that takes matrices as input and outputs matrices as follows: 

$$p(M) = a_0 I + a_1 M + \ldots + a_n M^n,$$ 

that is, each monomial $x^k$ is substituted by the corresponding matrix power $M^k$.

We say that a polynomial $p(x)$ annihilates a matrix $M\in\mathbb{R}^{n\times n}$ iff $p(M)=\boldsymbol{0}$, where $\boldsymbol{0}$ is the zero matrix.

**Task**

The task is to write a function ```annihilate_poly``` that takes as input an arbitrary square numpy array $M$ and outputs a vector whose cofficients are the coefficients of a (non-trivial) polynomial that annihilates $M$.  One-trivial means that its is not the zero polynomial which maps every matrix to the zero matrix.

**Hint**

You can reduce the problem to finding a linear dependance relationship between the $n+1$ vectors 

$$\mathrm{vec}(I), \mathrm{vec}(M), \mathrm{vec}(M^2),\ldots,\mathrm{vec}(M^n)\in\mathrm{R}^{n^2}.$$



The operation $\mathrm{vec}$ turns a square matrix $M\in\mathbb{R}^{n\times n}$ into a vector $v\in\mathbb{R}^{n^2}$ by first listing the entries of the first row, then those of the second row etc.

Update: 

To solve this problem, you have to compute the null space of the matrix $A\in \mathbb{R}^{n^2\times (n+1)}$ whose columns are the vectors $\mathrm{vec}(M^k)$ for $k\in\{0,\ldots,n\}$.


(This is not needed: 

If you don't remeber how to compute the find a linear dependance relationship, check out this stackoverflow post: https://math.stackexchange.com/questions/2198960/finding-linear-dependence-relation

You can use https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html to solve the resulting matrix equation.)

**Task**

Write a function ```annihilate_min_deg_poly``` that computes a non-trivial polynomial that annihilates a given square matrix and has the smallest possible degree.  Recall that a polynomial $p(x)$ has degree $d$ if the coefficient $a_{d+1}=\ldots=a_n=0$.

**Useful links:**

https://www.maths.ed.ac.uk/~tl/minimal.pdf

https://www.youtube.com/watch?v=3WkxhuzcSCE

https://github.com/asanaullah2015/cot5600



In [0]:
import numpy as np
from scipy.linalg import null_space
from numpy.linalg import LinAlgError

In [0]:
def annihilate_poly(M):
  #square numpy array M, n+1 vectors, k degree for polynomial we want to get
  M = np.matrix(M)
  n = M.shape[0]
  
  vector_pow = np.zeros((n**2, n+1))
  #print(vector_pow)
  for i in range(n+1):
    vector_pow[0:,i] = np.reshape((M**i),(n**2))

  n_space = null_space(vector_pow)
  return n_space

In [0]:
def find_min_degree(M, i, j, matrix_pow):
  
  M = np.matrix(M)
  n = M.shape[0]
  
  poly_deg = (i+j)//2
  degree_check = annihilate_poly(M)
  
  if i == j:
    n_space = annihilate_poly(M)
    return n_space[0:, 0]
  if degree_check.all == -1:
    return find_min_degree(M, poly_deg+1, j, matrix_pow)

  return find_min_degree(M, i, poly_deg, matrix_pow)


In [0]:
def annihilate_min_deg_poly(M,n):
  # computes a non-trivial polynomial that annihilates a given square matrix and has the smallest possible degree - we need to find it

  matrix_pow = np.zeros((n+1,n,n))
  matrix_pow[0] = np.identity(n) #vec(𝐼)

  #start from second and add powers vec(𝑀),vec(𝑀2),…,vec(𝑀𝑛)∈R𝑛2.
  for i in range(1,n+1):
    #matrix product of two arrays
    matrix_pow[i] = np.matmul(matrix_pow[i-1], M) 
  
  min_degree = find_min_degree(M, 0, n, matrix_pow)
  print('Minimum degree is ===>', np.prod(min_degree.shape) - 1, '\n')
  
  poly = 'Coeficients of polynomial that annihilates the matrix ====> \n \n'

  for i in range (0, np.prod(min_degree.shape) - 1):
    poly+= str(min_degree[i]) + "*x^" + str(i) + " + "

  poly+= str(min_degree[np.prod(min_degree.shape) - 1]) \
    + "*x^" + str(np.prod(min_degree.shape) - 1)
  
  print(poly, '\n')

  vector_pow = np.zeros((n**2, min_degree.size))

  for i in range (0, min_degree.size):
    vector_pow[0:,i] = np.reshape(matrix_pow[i], (n**2))
  
  final_coef = np.reshape(np.matmul(vector_pow, min_degree), (n,n))
  print('Solution using the coeficients  ===> \n')
  print(final_coef)

In [16]:
#matrix example from class, we know that min degree is 2

M = [[0,1],
     [1,0]]
n = 2

annihilate_min_deg_poly(M,n)
#annihilate_poly(M)


Minimum degree is ===> 2 

Coeficients of polynomial that annihilates the matrix ====> 
 
0.7071067811865475*x^0 + 0.0*x^1 + -0.7071067811865476*x^2 

Solution using the coeficients  ===> 

[[-1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16]]
